In [1]:
import glob
import json
import re

In [12]:
# 数据集中预处理过的数据所在文件夹
# base path of preprocessed data in the datasets
base_path = "../datasets/ds002799/derivatives/fmriprep/"

In [13]:
# 寻找被试在数据集中的id
# get subjects' id in datasets
subs = glob.glob(base_path+"/sub-*/")
print(subs[0])
reg_sub = re.compile(r"sub-\d+")
sub_ids = [reg_sub.search(i)[0] for i in subs]

In [2]:
# 从文件中读取每条数据的TR
# function that load repeation time for each run
def getTR(path):
    with open(path, "r") as f:
        jsonObj = json.load(f)
    return jsonObj["RepetitionTime"]

In [10]:
# 查找所需文件的路径等信息，主要包括：BOLD信号数据路径，TR重复时间，MASK遮罩文件路径，confounds混淆文件路径，
# search for every file's path and other info, such as BOLD signal path, TR repetition time, mask file path, confounds file path
datasets = {
    sub : {
        "ses-preop" : {
            "run-"+f"{i+1:0>2d}" : {
                "bold" : v,
                "mask" : glob.glob(base_path + sub + "/ses-preop/func/*run-" + f"{i+1:0>2d}" + "*MNI152*mask.nii.gz")[0],
                "confounds" : glob.glob(base_path + sub + "/ses-preop/func/*run-" + f"{i+1:0>2d}" + "*confounds_regressors.tsv")[0],
                "TR" : getTR(glob.glob(base_path + sub + "/ses-preop/func/*run-" + f"{i+1:0>2d}" + "*MNI152*preproc_bold.json")[0])
            } for i,v in enumerate(glob.glob(base_path + sub + "/ses-preop/func/*MNI152*preproc_bold.nii.gz"))
        },
        "ses-postop": {
            "run-"+f"{i+1:0>2d}" : {
                "bold" : v,
                "mask" : glob.glob(base_path + sub + "/ses-postop/func/*run-" + f"{i+1:0>2d}" + "*MNI152*mask.nii.gz")[0],
                "confounds" : glob.glob(base_path + sub + "/ses-postop/func/*run-" + f"{i+1:0>2d}" + "*confounds_regressors.tsv")[0],
                "TR" : getTR(glob.glob(base_path + sub + "/ses-postop/func/*run-" + f"{i+1:0>2d}" + "*MNI152*preproc_bold.json")[0])
            } for i,v in enumerate(glob.glob(base_path + sub + "/ses-postop/func/*MNI152*preproc_bold.nii.gz"))
        }
        } for sub in sub_ids
    }

In [11]:
# 保存为json文件
# save as json file
with open("datasets.json", "w") as f:
    json.dump(datasets, f, indent=4)